# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html
import json
# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
df_cityInfo = pd.read_csv("../output_data/cities.csv")
df_cityInfo

,CityID,City,Country,Latitude,Longitude,Temperature,MaxTemperature,Humidity,Wind Speed,Cloudiness
0,1,busselton,AU,-33.65,115.33,73.00,73.00,42,6.26,100
1,2,rikitea,PF,-23.12,-134.97,76.86,76.86,75,6.22,86
2,3,portoferraio,IT,42.81,10.31,68.97,75.20,45,11.41,5
3,4,barkly west,ZA,-28.54,24.52,82.40,82.40,58,13.87,20
4,5,provideniya,RU,64.38,-173.30,22.39,22.39,96,5.50,100
...,...,...,...,...,...,...,...,...,...,...
563,564,antalaha,MG,-14.90,50.28,83.44,83.44,67,14.12,15
564,565,villa carlos paz,AR,-31.42,-64.50,61.95,64.99,63,18.34,0
565,566,mau,IN,25.95,83.55,99.93,99.93,11,8.19,16
566,567,pyay,MM,18.82,95.22,92.53,92.53,28,6.24,58


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
locations = df_cityInfo[['Latitude', 'Longitude']]
weights = df_cityInfo['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig
#embed_minimal_html('export.html', views=[fig])

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
hotel_df = df_cityInfo.loc[(((df_cityInfo['Temperature'] < 80) & (df_cityInfo['Temperature'] > 70)) & (df_cityInfo['Wind Speed'] < 10)) & (df_cityInfo['Cloudiness'] == 0)]
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\smaheshw\AppData\Local\Continuum\anaconda3\envs\PythonData2020_1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CityID,City,Country,Latitude,Longitude,Temperature,MaxTemperature,Humidity,Wind Speed,Cloudiness,Hotel Name
265,266,verona,IT,45.42,11.03,76.80,81.00,23,4.70,0,
309,310,celano,IT,42.09,13.54,72.07,75.99,21,9.17,0,
426,427,port hedland,AU,-20.32,118.57,78.80,78.80,74,6.93,0,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="

In [15]:

for index, row in hotel_df.iterrows():
    lat=row["Latitude"]
    lon=row["Longitude"]
    search_url= base_url + str(lat) +","+str(lon)+"&radius=5000"+"&type=lodging"+"&keyword=hotel"+"&key="+ g_key
    response = requests.get(search_url).json()
    try:
        hotel_df.loc[index,'Hotel Name'] = response["results"][0]['name']
    except IndexError:
        hotel_df.loc[index,'Hotel Name']= ""
        print("No hotel found within 5000 meter radius")
hotel_df

C:\Users\smaheshw\AppData\Local\Continuum\anaconda3\envs\PythonData2020_1\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,CityID,City,Country,Latitude,Longitude,Temperature,MaxTemperature,Humidity,Wind Speed,Cloudiness,Hotel Name
265,266,verona,IT,45.42,11.03,76.80,81.00,23,4.70,0,Il Sogno di Giulietta
309,310,celano,IT,42.09,13.54,72.07,75.99,21,9.17,0,Il Fortino
426,427,port hedland,AU,-20.32,118.57,78.80,78.80,74,6.93,0,ibis Styles Port Hedland


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig
#embed_minimal_html('exporthotel.html', views=[fig])
# Display Map

Figure(layout=FigureLayout(height='420px'))